<a href="https://colab.research.google.com/github/ZAIN007777/Skin-Disease-Dectection-App/blob/main/skincare_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas faiss-cpu sentence-transformers openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [6]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load CSV file
csv_file = "dataset.csv"  # Ensure the correct filename
df = pd.read_csv(csv_file)

# Ensure correct column names
questions = df["question"].dropna().tolist()
answers = df["answer"].dropna().tolist()

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert questions into embeddings
question_embeddings = embedder.encode(questions, convert_to_numpy=True)

# Create FAISS index for fast similarity search
index = faiss.IndexFlatL2(question_embeddings.shape[1])
index.add(question_embeddings)

# Save index and data
faiss.write_index(index, "faiss_qa_index.bin")
np.save("qa_questions.npy", questions)
np.save("qa_answers.npy", answers)

print("Processed and stored questions and answers for retrieval.")


Processed and stored questions and answers for retrieval.


In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load stored data
index = faiss.read_index("faiss_qa_index.bin")
questions = np.load("qa_questions.npy", allow_pickle=True)
answers = np.load("qa_answers.npy", allow_pickle=True)

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_answer(question):
    # Convert question to embedding
    question_embedding = embedder.encode([question], convert_to_numpy=True)

    # Retrieve the closest matching question
    _, idx = index.search(question_embedding, k=1)
    retrieved_question = questions[idx[0][0]]
    retrieved_answer = answers[idx[0][0]]

    return f"Q: {retrieved_question}\nA: {retrieved_answer}"

# Example chatbot loop
while True:
    user_input = input("Ask a skincare question (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    print(get_answer(user_input))


Ask a skincare question (or type 'exit' to quit): how to have better skin?
Q: What are some skincare habits I should adopt for healthier skin?
A: Adopting a few key skincare habits can make a big difference in maintaining healthy skin:

1. Regular Cleansing: Clean your skin twice a day to remove dirt, oil, and makeup, ensuring your pores don't get clogged.

2. Moisturizing: Apply a moisturizer daily to keep your skin hydrated, which can prevent dryness and flaking.

3. Sun Protection: Use a broad-spectrum sunscreen with SPF 30 or higher every day, even when it's cloudy, to protect your skin from damaging UV rays.

4. Exfoliation: Exfoliate your skin 1-2 times a week to remove dead skin cells and rejuvenate your complexion.

5. Healthy Diet: Eat a balanced diet rich in fruits, vegetables, lean proteins, and healthy fats to provide your skin with the necessary nutrients.

6. Hydrate: Drink plenty of water throughout the day
Ask a skincare question (or type 'exit' to quit): clear skin
Q: 